<div class='alert alert-info' style='margin-top:20px'>
    <H1>SPACX LAUNCH PROJECT</H1>
</div>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import dash
from dash import html,dcc
from dash.dependencies import Input,Output

In [2]:
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv'
df = pd.read_csv(url)
print('Data is imported to data frame')

Data is imported to data frame


In [3]:
df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                56 non-null     int64  
 1   Flight Number             56 non-null     int64  
 2   Launch Site               56 non-null     object 
 3   class                     56 non-null     int64  
 4   Payload Mass (kg)         56 non-null     float64
 5   Booster Version           56 non-null     object 
 6   Booster Version Category  56 non-null     object 
dtypes: float64(1), int64(3), object(3)
memory usage: 3.2+ KB


In [65]:
# Create dashboard layout
app = dash.Dash(__name__)
app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard', style={'textAlign':'center', 'font-size':'40px'}),
    html.Div(dcc.Dropdown(id='site-dropdown', options=[
        {'label':'All sites', 'value':'All'},
        {'label':'CCAFS LC-40', 'value':'CCAFS LC-40'},
        {'label':'VAFB SLC-4E', 'value':'VAFB SLC-4E'},
        {'label':'KSC LC-39A', 'value':'KSC LC-39A'},
        {'label':'CCAFS SLC-40', 'value':'CCAFS SLC-40'}],
        placeholder='Select launch site(s)',
        value='All',
        searchable=True),
        style={'font-size':'20px', 'padding':'2px'}),
    html.Br(),
    html.Br(),
    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),
    html.Br(),
    html.Div(dcc.RangeSlider(id='payload-slider', min=0, max=10000, step=1000, value=[0,1000])),
    html.Div(dcc.Graph(id='success-payload'))
])
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def success_launch_bySites(site):
    if site=='All':
        df_pie = df.groupby('Launch Site')['class'].sum().reset_index()
        fig = px.pie(df_pie, values='class', names='Launch Site', title='Total success launch by All Sites')
    else:
        df_pie = df[df['Launch Site'] == site]['class'].value_counts().reset_index()
        fig = px.pie(df_pie, values='count', names=['Unsuccess', 'Success'], 
                     title='Total success launch by site {}'.format(site))
    return fig

@app.callback(
    Output('success-payload', 'figure'),
    [Input('site-dropdown', 'value'), Input('payload-slider', 'value')]
)
def success_launch_byPayLoad(site, payload):
    if site=='All':
        fig = px.scatter(df, x='Payload Mass (kg)', y='class', color='Booster Version Category')
    else:
        df_scatter = df[(df['Launch Site']==site) & [x in range(payload[0], payload[1]) for x in df['Payload Mass (kg)']]]
        fig = px.scatter(df_scatter, x='Payload Mass (kg)', y='class', color='Booster Version Category')
    return fig
    
if __name__ == '__main__':
    app.run_server(port=8005, host='127.0.0.1', debug=True)
                                              